In [69]:
# -*- coding: utf-8 -*-
"""
Script to process images recursively using Google Gemini Vision API for emotion annotation.

NEW STRATEGY (v6 - Prompt Enhancement):
- Configuration moved to the top (API Key, Model, Input Folder, Threads).
- Processes images in an input folder and all its subfolders using multiple threads.
- Skips processing an image if its corresponding JSON file already exists AND
  contains an annotation entry for the currently specified MODEL_NAME.
- Appends the new annotation to the existing list in the JSON if other model
  annotations are present. Creates a new JSON list if the file doesn't exist.
- Uses a revised prompt asking for ratings (with constraints) and reasoning for
  40 standard emotions PLUS 4 extra dimensions (Arousal, Valence, Dominance, Emotional Vulnerability).
- **Prompt Enhancement:** Further strengthened the instructions regarding the
  "Top 5 Non-Zero" constraint for standard emotions to improve model adherence.
- Top-5 Constraint: For the 40 standard emotions, only the top 5 most perceived
  can have non-zero ratings (0-7 scale). ALL others MUST be 0.
- Extra Dimensions: Rated on specific scales (Arousal 0-7, Valence -3 to +3,
  Dominance -3 to +3, Emotional Vulnerability 0-5) without the top-5 constraint.
- LLM Output Expectation: A single dictionary with keys "emotions" and "extra_dimensions".
  Each contains dimension names mapping to {"rating": N, "reasoning": "..."}.
- Final JSON Output: Saves/Updates a JSON file *next to* each image (image.json for image.ext).
  The JSON root is a LIST containing annotation dictionaries:
  [{"model": "some_model", "predictions": {...}}, {"model": MODEL_NAME, "predictions": {...}}]
- Includes detailed multi-shot examples and definitions in the prompt.
- Retains robust validation, retry logic, cleaning, and error handling within each worker thread.
"""

import os
import json
import requests
import base64
import mimetypes
import time
import ast
import traceback
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

import ast
import json
import re

# --- User Configuration ---

# !!! IMPORTANT: Replace with your actual hyperlan API Key !!!
API_KEY = ""  # <--- REPLACE THIS (Using key from user's previous code)

# Specify the  model to use for *this run*
MODEL_NAME = "claude-3-7-sonnet-latest" #"gpt-4o" #"nova-pro" # <--- RECOMMEND using latest flash model for potentially better adherence

# Folder where your input images are stored (will be scanned recursively)
INPUT_FOLDER = '' # Example input folder (Using path from user's logs)
OUTPUT_FOLDER = ""  # Change as needed
# Number of parallel threads to use for API calls
NUM_THREADS = 50

# --- Script Constants ---

# Define supported image file extensions
SUPPORTED_EXTENSIONS = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.webp')

# Define the 40 standard emotion dimensions and their descriptions
EMOTION_DIMENSIONS = {
    "Amusement": "'lighthearted fun', 'mirth', 'joviality', 'laughter', 'playfulness'",
    "Elation": "'happiness', 'excitement', 'joy', 'exhilaration', 'delight', 'jubilation'",
    "Pleasure/Ecstasy": "'ecstasy', 'pleasure', 'bliss', 'rapture', 'Beatitude'",
    "Contentment": "'contentment', 'relaxation', 'peacefulness', 'calmness', 'satisfaction', 'Ease'",
    "Thankfulness/Gratitude": "'thankfulness', 'gratitude', 'appreciation'",
    "Affection": "'sympathy', 'compassion', 'warmth', 'trust', 'caring', 'Tenderness'",
    "Infatuation": "'infatuation', 'crush', 'romantic desire', 'fondness', 'adoration'",
    "Hope/Enthusiasm/Optimism": "'hope', 'enthusiasm', 'optimism', 'Anticipation', 'Courage'",
    "Triumph": "'triumph', 'superiority', 'victory'",
    "Pride": "'pride', 'dignity', 'self-confidently', 'honor'",
    "Interest": "'interest', 'fascination', 'curiosity', 'intrigue'",
    "Awe": "'awe', 'awestruck', 'wonder', 'amazement'",
    "Astonishment/Surprise": "'astonishment', 'surprise', 'shock', 'startlement'",
    "Concentration": "'concentration', 'deep focus', 'engrossment', 'absorption'",
    "Contemplation": "'contemplation', 'thoughtfulness', 'pondering', 'reflection', 'meditation'",
    "Relief": "'relief', 'respite', 'alleviation', 'solace', 'comfort'",
    "Longing": "'yearning', 'longing', 'pining', 'wistfulness', 'nostalgia', 'desire'",
    "Teasing": "'teasing', 'bantering', 'mocking playfully', 'provoking lightly'",
    "Impatience and Irritability": "'impatience', 'irritability', 'restlessness', 'exasperation'",
    "Sexual Lust": "'sexual lust', 'carnal desire', 'libido', 'feeling turned on'",
    "Doubt": "'doubt', 'distrust', 'suspicion', 'skepticism', 'uncertainty'",
    "Fear": "'fear', 'terror', 'dread', 'apprehension', 'alarm', 'horror', 'panic'",
    "Distress": "'worry', 'anxiety', 'unease', 'anguish', 'trepidation', 'Concern'",
    "Confusion": "'confusion', 'bewilderment', 'flabbergasted', 'disorientation', 'Perplexity'",
    "Embarrassment": "'embarrassment', 'shyness', 'mortification', 'awkwardness', 'Self-Consciousness'",
    "Shame": "'shame', 'guilt', 'remorse', 'humiliation', 'contrition'",
    "Disappointment": "'disappointment', 'regret', 'dismay', 'letdown'",
    "Sadness": "'sadness', 'sorrow', 'grief', 'melancholy', 'Dejection', 'Despair'",
    "Bitterness": "'resentment', 'acrimony', 'bitterness', 'cynicism'",
    "Contempt": "'contempt', 'disapproval', 'scorn', 'disdain', 'loathing'",
    "Disgust": "'disgust', 'revulsion', 'repulsion', 'abhorrence'",
    "Anger": "'anger', 'rage', 'fury', 'hate', 'irascibility', 'Wrath'",
    "Malevolence/Malice": "'spite', 'sadism', 'malevolence', 'malice', 'schadenfreude'",
    "Sourness": "'sourness', 'tartness', 'acerbic attitude'",
    "Pain": "'physical pain', 'suffering', 'torment', 'ache'",
    "Helplessness": "'helplessness', 'powerlessness', 'desperation', 'submission'",
    "Fatigue/Exhaustion": "'fatigue', 'exhaustion', 'weariness', 'lethargy', 'burnout'",
    "Emotional Numbness": "'numbness', 'detachment', 'insensitivity', 'apathy', 'boredom', 'indifference'",
    "Intoxication/Altered States of Consciousness": "'being drunk', 'stupor', 'disorientation', 'altered perception'",
    "Jealousy & Envy": "'jealousy', 'envy', 'covetousness'"
}
REQUIRED_EMOTION_KEYS = set(EMOTION_DIMENSIONS.keys())
taxonomy_list_string = "\n".join([f"- **{name}**: {desc}" for name, desc in EMOTION_DIMENSIONS.items()])

# Define the 4 extra dimensions
EXTRA_DIMENSIONS = {
    "Arousal": {"description": "Physiological state of being awoken or stimulated.", "scale": "0-7", "levels": {0: "Asleep", 1: "Very calm", 2: "Calm", 3: "Neutral baseline", 4: "Engaged", 5: "Activated", 6: "Highly aroused", 7: "Extremely aroused"}},
    "Valence": {"description": "Intrinsic attractiveness (positive) or aversiveness (negative).", "scale": "-3 to +3", "levels": {-3: "Extremely negative", -2: "Strongly negative", -1: "Mildly negative", 0: "Neutral", 1: "Mildly positive", 2: "Strongly positive", 3: "Extremely positive"}},
    "Dominance": {"description": "Degree of control, influence, assertiveness vs. submissiveness.", "scale": "-3 to +3", "levels": {-3: "Extremely submissive", -2: "Strongly submissive", -1: "Mildly submissive", 0: "Neutral", 1: "Mildly dominant", 2: "Strongly dominant", 3: "Extremely dominant"}},
    "Emotional Vulnerability": {"description": "Degree of emotional openness, susceptibility vs. guardedness.", "scale": "0-5", "levels": {0: "Extremely guarded", 1: "Strongly guarded", 2: "Moderately guarded", 3: "Slightly open", 4: "Clearly open", 5: "Extremely open/vulnerable"}}
}
REQUIRED_EXTRA_DIMENSION_KEYS = set(EXTRA_DIMENSIONS.keys())

# Format extra dimension definitions for the prompt (using original full definitions)
extra_dims_string = """
**Arousal (0-7)**
*Description*: Physiological and psychological state of being awoken or of sense organs stimulated to a point of perception.
*Scale Levels*:
  * 0: Completely calm, asleep, inert.
  * 1: Very calm, meditative, extremely relaxed but awake.
  * 2: Calm, relaxed, low energy.
  * 3: Slightly activated, neutral baseline awareness.
  * 4: Moderately aroused, engaged, attentive.
  * 5: Clearly aroused, alert, activated (e.g., moderate excitement, mild stress).
  * 6: Highly aroused, very alert, strong activation (e.g., strong joy, anger, fear, surprise).
  * 7: Extremely aroused, peak activation, hyper-alert, overwhelmed (e.g., intense rage, terror, ecstasy, shock).

**Valence (-3 to +3)**
*Description*: The intrinsic attractiveness (positive valence) or aversiveness (negative valence) of an event, object, or situation.
*Scale Levels*:
  * -3: Extremely negative (e.g., intense despair, rage, disgust, terror).
  * -2: Strongly negative (e.g., significant sadness, anger, fear, contempt).
  * -1: Mildly negative (e.g., slight annoyance, disappointment, worry).
  * 0: Neutral (e.g., calm, indifferent, neutral surprise, concentration).
  * 1: Mildly positive (e.g., slight contentment, interest, pleasantness).
  * 2: Strongly positive (e.g., clear happiness, affection, gratitude, relief).
  * 3: Extremely positive (e.g., intense joy, elation, ecstasy, deep love, awe).

**Dominance (-3 to +3)**
*Description*: The degree to which the person appears to be in control, influential, and assertive versus controlled, influenced, and submissive.
*Scale Levels*:
  * -3: Extremely submissive (e.g., appearing completely powerless, helpless, yielding, crushed).
  * -2: Strongly submissive (e.g., clearly deferential, meek, intimidated).
  * -1: Mildly submissive (e.g., appearing slightly hesitant, shy, yielding).
  * 0: Neutral (e.g., appearing neither particularly dominant nor submissive, balanced).
  * 1: Mildly dominant (e.g., appearing slightly confident, assertive, leading).
  * 2: Strongly dominant (e.g., clearly assertive, confident, in control, leading).
  * 3: Extremely dominant (e.g., appearing commanding, highly controlling, imposing, powerful).

**Emotional Vulnerability (0-5)**
*Description*: The degree to which the person appears emotionally open, susceptible to emotional influence, fragile, or unguarded versus closed off, resilient, numb, or guarded.
*Scale Levels*:
  * 0: Extremely guarded/invulnerable (e.g., appearing completely numb, apathetic, cold, emotionally shut down, uninfluenceable).
  * 1: Strongly guarded (e.g., stoic, detached, emotionally reserved, resistant to influence).
  * 2: Moderately guarded (e.g., controlled expression, neutral, keeping composure).
  * 3: Slightly open/vulnerable (e.g., showing subtle signs of emotion, slightly receptive).
  * 4: Clearly open/vulnerable (e.g., expressing genuine emotion openly, whether positive like warmth/joy or negative like sadness/grief, appears influenceable/sensitive).
  * 5: Extremely open/vulnerable (e.g., completely emotionally exposed, raw, fragile, outpouring of feeling, deeply affected, heart-on-sleeve).
"""

# --- Multi-Shot Examples ---
MULTI_SHOT_EXAMPLES = """
**MULTI-SHOT EXAMPLES OF EXPECTED OUTPUT:**

*Example 1: Image shows a person laughing heartily at a joke.*

{
    "emotions": {
        "Amusement": {"rating": 7, "reasoning": "Wide open mouth smile, eyes crinkled (Duchenne marker), head tilted back slightly, indicative of strong, genuine amusement and laughter."},
        "Elation": {"rating": 5, "reasoning": "High positive energy apparent from the strong smile and activated facial muscles, suggesting significant joy beyond just amusement."},
        "Pleasure/Ecstasy": {"rating": 3, "reasoning": "The laughter suggests a moment of intense pleasure, though perhaps not full ecstasy. Cheeks are raised."},
        "Contentment": {"rating": 2, "reasoning": "While elated, there's an underlying sense of ease and satisfaction suggested by the uninhibited laughter."},
        "Affection": {"rating": 1, "reasoning": "Depending on context (e.g., laughing *with* someone), slight warmth might be inferred, but it's weak based on face alone. Eyes are focused on the source of humor."},
        "Thankfulness/Gratitude": {"rating": 0, "reasoning": "No visual cues like clasped hands, specific gaze, or bowing suggest gratitude."},
        "Infatuation": {"rating": 0, "reasoning": "Expression is broad amusement, not the focused gaze or softer smile often linked to infatuation."},
        "Hope/Enthusiasm/Optimism": {"rating": 0, "reasoning": "While positive, the expression is reactive amusement, not forward-looking hope or specific enthusiasm."},
        "Triumph": {"rating": 0, "reasoning": "No signs of superiority or victory; expression is purely mirthful."},
        "Pride": {"rating": 0, "reasoning": "Posture and expression don't suggest pride (e.g., puffed chest, chin up). Focus is external."},
        "Interest": {"rating": 0, "reasoning": "While engaged, the primary emotion is amusement, not curiosity or focused interest."},
        "Awe": {"rating": 0, "reasoning": "Expression lacks the wide eyes and dropped jaw often associated with awe."},
        "Astonishment/Surprise": {"rating": 0, "reasoning": "Features are consistent with laughter, not the sudden intake of breath or wide-eyed shock of surprise."},
        "Concentration": {"rating": 0, "reasoning": "Expression is outward and reactive, opposite of inward focus or concentration."},
        "Contemplation": {"rating": 0, "reasoning": "Facial expression is active and expressive, not thoughtful or meditative."},
        "Relief": {"rating": 0, "reasoning": "No visual cues suggesting release from tension or worry."},
        "Longing": {"rating": 0, "reasoning": "Gaze and expression are present-focused amusement, not wistful or yearning."},
        "Teasing": {"rating": 0, "reasoning": "Could be response to teasing, but the expression itself isn't teasing (e.g., no playful smirk or wink)."},
        "Impatience and Irritability": {"rating": 0, "reasoning": "Facial features are relaxed and positive, contrary to signs of irritation."},
        "Sexual Lust": {"rating": 0, "reasoning": "Expression is generalized amusement, lacks cues for specific sexual desire."},
        "Doubt": {"rating": 0, "reasoning": "Expression conveys certainty of amusement, not skepticism or uncertainty."},
        "Fear": {"rating": 0, "reasoning": "Features are open and positive, opposite of fear cues (e.g., wide eyes, tense mouth)."},
        "Distress": {"rating": 0, "reasoning": "Strong positive emotion, no signs of worry or anxiety."},
        "Confusion": {"rating": 0, "reasoning": "Expression is clear amusement, not disorientation or perplexity."},
        "Embarrassment": {"rating": 0, "reasoning": "Open laughter, no signs of shrinking, blushing, or averted gaze associated with embarrassment."},
        "Shame": {"rating": 0, "reasoning": "Expression is open and positive, contrary to shame cues (e.g., slumped posture, downcast eyes)."},
        "Disappointment": {"rating": 0, "reasoning": "Features are uplifted, opposite of cues for disappointment (e.g., downturned mouth)."},
        "Sadness": {"rating": 0, "reasoning": "Strong positive affect, opposite of sadness cues."},
        "Bitterness": {"rating": 0, "reasoning": "Open, genuine smile, lacks the tight lips or narrowed eyes of bitterness."},
        "Contempt": {"rating": 0, "reasoning": "Symmetrical smile, no lip corner raise or sneer indicative of contempt."},
        "Disgust": {"rating": 0, "reasoning": "Expression is inviting, opposite of revulsion cues (e.g., nose wrinkle, upper lip raise)."},
        "Anger": {"rating": 0, "reasoning": "Relaxed brow, open smile, opposite of anger cues (e.g., furrowed brow, tense jaw)."},
        "Malevolence/Malice": {"rating": 0, "reasoning": "Genuine positive expression, no hint of spite or ill will."},
        "Sourness": {"rating": 0, "reasoning": "Facial expression is warm and amused, not acerbic or tart."},
        "Pain": {"rating": 0, "reasoning": "Expression indicates pleasure, not physical discomfort."},
        "Helplessness": {"rating": 0, "reasoning": "Expression is active and engaged, not passive or powerless."},
        "Fatigue/Exhaustion": {"rating": 0, "reasoning": "High energy level apparent, contrary to signs of tiredness."},
        "Emotional Numbness": {"rating": 0, "reasoning": "Highly expressive and emotional, opposite of numbness or detachment."},
        "Intoxication/Altered States of Consciousness": {"rating": 0, "reasoning": "Expression seems clear and situationally appropriate, no obvious signs of altered state."},
        "Jealousy & Envy": {"rating": 0, "reasoning": "Open positive expression, no narrowed eyes or sideways glances suggesting jealousy."}
        # (INFO: 5 non-zero ratings - constraint met: Amusement, Elation, Pleasure/Ecstasy, Contentment, Affection)
    },
    "extra_dimensions": {
        "Arousal": {"rating": 6, "reasoning": "High energy level evident in the hearty laughter, wide smile, and activated facial muscles."},
        "Valence": {"rating": 3, "reasoning": "Extremely positive emotion expressed through clear joy and amusement."},
        "Dominance": {"rating": 1, "reasoning": "Appears confident and uninhibited in expressing emotion, suggesting mild dominance or at least lack of submissiveness in the moment."},
        "Emotional Vulnerability": {"rating": 4, "reasoning": "The open, genuine expression of strong positive emotion suggests a high degree of emotional openness and lack of guardedness in this context."}
    }
}


*Example 2: Image shows a person looking stern and focused, perhaps concentrating on a task.*

{
    "emotions": {
        "Concentration": {"rating": 6, "reasoning": "Eyes are narrowed slightly and intensely focused, brow might be slightly furrowed, lips pressed together lightly, indicating deep mental engagement."},
        "Contemplation": {"rating": 2, "reasoning": "While focused, the intensity suggests active concentration more than passive pondering. Some slight thoughtfulness might be present."},
        "Interest": {"rating": 4, "reasoning": "The focused gaze clearly indicates interest in the subject of concentration."},
        "Hope/Enthusiasm/Optimism": {"rating": 3, "reasoning": "Determination for the task is implied by the focused look and set jaw; mapping 'Determination' here as per taxonomy."},
        "Impatience and Irritability": {"rating": 1, "reasoning": "Slight tension in the brow and mouth *could* hint at minor impatience if the task is difficult, but concentration is dominant."},
        "Amusement": {"rating": 0, "reasoning": "Face is serious and focused, lacking any cues of mirth or playfulness."},
        "Elation": {"rating": 0, "reasoning": "Expression is neutral-to-serious, lacking signs of joy or excitement."},
        "Pleasure/Ecstasy": {"rating": 0, "reasoning": "No visible signs of pleasure; expression is work-oriented."},
        "Contentment": {"rating": 0, "reasoning": "Face shows tension of focus, not relaxation or contentment."},
        "Thankfulness/Gratitude": {"rating": 0, "reasoning": "Expression is task-focused, not indicative of gratitude."},
        "Affection": {"rating": 0, "reasoning": "Expression lacks warmth or connection cues."},
        "Infatuation": {"rating": 0, "reasoning": "Focus appears task-related, not romantic."},
        "Triumph": {"rating": 0, "reasoning": "No signs of victory or superiority."},
        "Pride": {"rating": 0, "reasoning": "Expression is focused, not specifically proud."},
        "Awe": {"rating": 0, "reasoning": "No signs of wonder or amazement."},
        "Astonishment/Surprise": {"rating": 0, "reasoning": "Expression is focused, not shocked."},
        "Relief": {"rating": 0, "reasoning": "Expression shows tension, not release."},
        "Longing": {"rating": 0, "reasoning": "Focus is present, not wistful."},
        "Teasing": {"rating": 0, "reasoning": "Expression is serious, not playful."},
        "Sexual Lust": {"rating": 0, "reasoning": "Expression is work-focused, not desirous."},
        "Doubt": {"rating": 0, "reasoning": "Expression conveys focus, not uncertainty."},
        "Fear": {"rating": 0, "reasoning": "No signs of fear present."},
        "Distress": {"rating": 0, "reasoning": "Focus is intense, but not necessarily anxious or worried."},
        "Confusion": {"rating": 0, "reasoning": "Expression is focused, not perplexed."},
        "Embarrassment": {"rating": 0, "reasoning": "Expression is confident/neutral, not shy or awkward."},
        "Shame": {"rating": 0, "reasoning": "No cues of guilt or remorse."},
        "Disappointment": {"rating": 0, "reasoning": "Expression is neutral/focused, not let down."},
        "Sadness": {"rating": 0, "reasoning": "No downturned mouth or other sadness cues; expression is focused."},
        "Bitterness": {"rating": 0, "reasoning": "No signs of resentment."},
        "Contempt": {"rating": 0, "reasoning": "No sneer or disdain visible."},
        "Disgust": {"rating": 0, "reasoning": "No signs of revulsion."},
        "Anger": {"rating": 0, "reasoning": "While potentially intense, lacks specific anger markers like flared nostrils or deep brow furrow directed at a person."},
        "Malevolence/Malice": {"rating": 0, "reasoning": "No hint of spite or ill will."},
        "Sourness": {"rating": 0, "reasoning": "Expression is neutral/focused, not acerbic."},
        "Pain": {"rating": 0, "reasoning": "No signs of physical discomfort."},
        "Helplessness": {"rating": 0, "reasoning": "Expression conveys agency, not powerlessness."},
        "Fatigue/Exhaustion": {"rating": 0, "reasoning": "Eyes are alert and focused, not weary."},
        "Emotional Numbness": {"rating": 0, "reasoning": "Expression shows engagement, not detachment."},
        "Intoxication/Altered States of Consciousness": {"rating": 0, "reasoning": "Appearance is clear and focused."},
        "Jealousy & Envy": {"rating": 0, "reasoning": "Focus appears directed at a task, not involving social comparison."}
        # (INFO: 5 non-zero ratings - constraint met: Concentration, Contemplation, Interest, Hope/Enthusiasm/Optimism, Impatience and Irritability)
    },
    "extra_dimensions": {
        "Arousal": {"rating": 4, "reasoning": "Moderately aroused state required for active concentration and mental effort. Alert and engaged."},
        "Valence": {"rating": 0, "reasoning": "The expression is primarily neutral, focused on the task rather than expressing strong positive or negative feeling."},
        "Dominance": {"rating": 1, "reasoning": "The focused and controlled expression suggests a sense of agency and mild dominance over the task or situation."},
        "Emotional Vulnerability": {"rating": 1, "reasoning": "Appears focused and potentially guarded due to concentration, low emotional openness displayed."}
    }
}

*Example 3: Image shows a person looking down with a slight frown, appearing sad or disappointed.*

{
    "emotions": {
        "Sadness": {"rating": 5, "reasoning": "Eyes are downcast, corners of the lips are slightly downturned, brow may show slight tension, overall posture (if visible) might be slumped."},
        "Disappointment": {"rating": 4, "reasoning": "The sadness seems linked to a specific unmet expectation or outcome, fitting disappointment. Mouth corners down."},
        "Contemplation": {"rating": 2, "reasoning": "The inward, downcast gaze suggests some level of reflection or brooding alongside the sadness."},
        "Helplessness": {"rating": 1, "reasoning": "A hint of powerlessness might be inferred from the downcast gaze and lack of assertive posture, but it's weak."},
        "Fatigue/Exhaustion": {"rating": 1, "reasoning": "Low energy might contribute to the expression, potentially slight bags under eyes, but primary cue is sadness."},
        "Amusement": {"rating": 0, "reasoning": "Expression is clearly negative and subdued, opposite of amusement."},
        "Elation": {"rating": 0, "reasoning": "Lack of positive affect, features are downturned."},
        "Pleasure/Ecstasy": {"rating": 0, "reasoning": "No indication of pleasure."},
        "Contentment": {"rating": 0, "reasoning": "Expression lacks peace or satisfaction cues."},
        "Thankfulness/Gratitude": {"rating": 0, "reasoning": "No signs of gratitude."},
        "Affection": {"rating": 0, "reasoning": "Lacks warmth or connection cues."},
        "Infatuation": {"rating": 0, "reasoning": "Expression not indicative of romantic interest."},
        "Hope/Enthusiasm/Optimism": {"rating": 0, "reasoning": "Expression suggests lack of hope, downcast rather than forward-looking."},
        "Triumph": {"rating": 0, "reasoning": "No signs of victory."},
        "Pride": {"rating": 0, "reasoning": "Expression lacks pride cues."},
        "Interest": {"rating": 0, "reasoning": "Gaze is inward/down, not actively curious."},
        "Awe": {"rating": 0, "reasoning": "No signs of wonder."},
        "Astonishment/Surprise": {"rating": 0, "reasoning": "No signs of shock."},
        "Concentration": {"rating": 0, "reasoning": "Expression is sad/contemplative, not focused effort."},
        "Relief": {"rating": 0, "reasoning": "No signs of released tension."},
        "Longing": {"rating": 0, "reasoning": "While sad, lacks specific yearning cues distinct from general sadness."},
        "Teasing": {"rating": 0, "reasoning": "Expression is withdrawn, not playful."},
        "Impatience and Irritability": {"rating": 0, "reasoning": "Expression is low energy sadness, not irritation."},
        "Sexual Lust": {"rating": 0, "reasoning": "Expression lacks desire cues."},
        "Doubt": {"rating": 0, "reasoning": "Expression is sadness, not uncertainty."},
        "Fear": {"rating": 0, "reasoning": "No fear cues like wide eyes or tension."},
        "Distress": {"rating": 0, "reasoning": "While sad, lacks acute anxiety or worry cues distinct from sadness."},
        "Confusion": {"rating": 0, "reasoning": "Expression is comprehensible sadness, not bewilderment."},
        "Embarrassment": {"rating": 0, "reasoning": "No cues of shyness or awkwardness."},
        "Shame": {"rating": 0, "reasoning": "While negative, lacks specific guilt/humiliation cues distinct from sadness."},
        "Bitterness": {"rating": 0, "reasoning": "Expression is sad, not resentful."},
        "Contempt": {"rating": 0, "reasoning": "No sneer or disdain."},
        "Disgust": {"rating": 0, "reasoning": "No revulsion cues."},
        "Anger": {"rating": 0, "reasoning": "Expression lacks the tension, narrowed eyes, or outward energy of anger."},
        "Malevolence/Malice": {"rating": 0, "reasoning": "No hint of spite."},
        "Sourness": {"rating": 0, "reasoning": "Expression is sad, not acerbic."},
        "Pain": {"rating": 0, "reasoning": "No grimacing or physical pain cues."},
        "Emotional Numbness": {"rating": 0, "reasoning": "Expresses clear emotion (sadness), not numbness."},
        "Intoxication/Altered States of Consciousness": {"rating": 0, "reasoning": "Appearance seems normal."},
        "Jealousy & Envy": {"rating": 0, "reasoning": "No cues related to social comparison."},
        # (INFO: 5 non-zero ratings - constraint met: Sadness, Disappointment, Contemplation, Helplessness, Fatigue/Exhaustion)
    },
    "extra_dimensions": {
        "Arousal": {"rating": 2, "reasoning": "Low energy state associated with sadness and potential fatigue. Calm but negatively toned."},
        "Valence": {"rating": -2, "reasoning": "Strongly negative emotion is clearly conveyed through the sad and disappointed expression."},
        "Dominance": {"rating": -1, "reasoning": "The potentially slumped posture and downcast gaze suggest a lack of control or influence, leaning towards mild submissiveness."},
        "Emotional Vulnerability": {"rating": 4, "reasoning": "The visible sadness suggests a state of emotional openness and sensitivity to the negative feeling. Appears affected."}
    }
}

*Example 4: Image shows a person with wide eyes, raised eyebrows, and slightly open mouth, looking surprised.*
{
    "emotions": {
        "Astonishment/Surprise": {"rating": 7, "reasoning": "Classic surprise markers: Eyes wide open, eyebrows significantly raised, jaw slightly dropped, indicating a strong reaction to something unexpected."},
        "Interest": {"rating": 3, "reasoning": "The surprise implies a sudden shift of attention and interest towards the surprising stimulus."},
        "Awe": {"rating": 2, "reasoning": "If the surprise is towards something impressive, a slight element of awe might be present in the wide-eyed look."},
        "Fear": {"rating": 1, "reasoning": "Depending on the nature of the surprise, a very mild, brief fear response (startle) might be part of the initial reaction, seen in the wide eyes."},
        "Confusion": {"rating": 1, "reasoning": "A surprising event can momentarily cause confusion; the wide eyes might briefly reflect trying to process the unexpected."},
        "Amusement": {"rating": 0, "reasoning": "Expression is shock/surprise, not mirth. Mouth is open from surprise, not smiling."},
        "Elation": {"rating": 0, "reasoning": "While intense, the expression is shock, not necessarily positive joy."},
        "Pleasure/Ecstasy": {"rating": 0, "reasoning": "Expression is shock, not pleasure."},
        "Contentment": {"rating": 0, "reasoning": "Expression is activated surprise, not calm contentment."},
        "Thankfulness/Gratitude": {"rating": 0, "reasoning": "No gratitude cues."},
        "Affection": {"rating": 0, "reasoning": "Expression lacks warmth."},
        "Infatuation": {"rating": 0, "reasoning": "Expression is shock, not romantic interest."},
        "Hope/Enthusiasm/Optimism": {"rating": 0, "reasoning": "Expression is reactive shock, not forward-looking."},
        "Triumph": {"rating": 0, "reasoning": "No victory cues."},
        "Pride": {"rating": 0, "reasoning": "No pride cues."},
        "Concentration": {"rating": 0, "reasoning": "Expression is outward shock, not inward focus."},
        "Contemplation": {"rating": 0, "reasoning": "Expression is reactive, not thoughtful."},
        "Relief": {"rating": 0, "reasoning": "No cues of released tension."},
        "Longing": {"rating": 0, "reasoning": "Expression is immediate shock, not yearning."},
        "Teasing": {"rating": 0, "reasoning": "Expression is receiving surprise, not teasing."},
        "Impatience and Irritability": {"rating": 0, "reasoning": "Expression is shock, not irritation."},
        "Sexual Lust": {"rating": 0, "reasoning": "Expression lacks desire cues."},
        "Doubt": {"rating": 0, "reasoning": "Expression is shock, not skepticism."},
        "Distress": {"rating": 0, "reasoning": "While potentially startling (related to fear=1), lacks specific worry/anguish cues beyond surprise itself."},
        "Embarrassment": {"rating": 0, "reasoning": "Expression is open shock, not shyness."},
        "Shame": {"rating": 0, "reasoning": "No guilt cues."},
        "Disappointment": {"rating": 0, "reasoning": "Features are lifted/neutral, not downturned."},
        "Sadness": {"rating": 0, "reasoning": "Facial features are lifted and activated, opposite of sadness."},
        "Bitterness": {"rating": 0, "reasoning": "No resentment cues."},
        "Contempt": {"rating": 0, "reasoning": "No disdain cues."},
        "Disgust": {"rating": 0, "reasoning": "No revulsion cues."},
        "Anger": {"rating": 0, "reasoning": "Expression lacks hostility cues; features suggest shock."},
        "Malevolence/Malice": {"rating": 0, "reasoning": "No spite cues."},
        "Sourness": {"rating": 0, "reasoning": "Expression is shock, not acerbic."},
        "Pain": {"rating": 0, "reasoning": "No pain cues."},
        "Helplessness": {"rating": 0, "reasoning": "While potentially losing control momentarily, lacks specific helplessness cues."},
        "Fatigue/Exhaustion": {"rating": 0, "reasoning": "Expression is highly alert, not tired."},
        "Emotional Numbness": {"rating": 0, "reasoning": "Expression is highly reactive, not numb."},
        "Intoxication/Altered States of Consciousness": {"rating": 0, "reasoning": "Appears situationally appropriate shock."},
        "Jealousy & Envy": {"rating": 0, "reasoning": "No social comparison cues."},
        # (INFO: 5 non-zero ratings - constraint met: Astonishment/Surprise, Interest, Awe, Fear, Confusion)
    },
    "extra_dimensions": {
        "Arousal": {"rating": 6, "reasoning": "High physiological arousal indicated by the strong, sudden reaction: wide eyes, raised brows, activated state."},
        "Valence": {"rating": 0, "reasoning": "Surprise itself is relatively neutral valence initially; it depends on what caused it. The expression alone doesn't guarantee positive or negative feeling."},
        "Dominance": {"rating": -1, "reasoning": "Being surprised often implies momentarily losing control or composure, suggesting a brief shift towards mild submissiveness to the event."},
        "Emotional Vulnerability": {"rating": 3, "reasoning": "The unguarded, reactive expression of surprise shows a moderate level of emotional openness in the moment."}
    }
}

*Example 5: Image shows a person with a neutral, calm expression, looking directly at the camera.*
{
    "emotions": {
        "Contentment": {"rating": 1, "reasoning": "A very subtle sense of calm or ease might be inferred from the relaxed neutral features, but it's extremely weak."},
        "Emotional Numbness": {"rating": 1, "reasoning": "The lack of clear expression could be interpreted as slight detachment or indifference, but could also just be neutral."},
        # The following 3 are rated 0 because there are no positive indicators, and we need to keep non-zero count <= 5.
        "Amusement": {"rating": 0, "reasoning": "No smile, eye crinkling, or other signs of amusement. Facial muscles appear relaxed."},
        "Elation": {"rating": 0, "reasoning": "Lack of visible positive energy, smile, or bright eyes."},
        "Pleasure/Ecstasy": {"rating": 0, "reasoning": "Neutral expression provides no indication of pleasure."},
        "Thankfulness/Gratitude": {"rating": 0, "reasoning": "No specific cues for gratitude present in the neutral face."},
        "Affection": {"rating": 0, "reasoning": "Gaze is direct but lacks specific warmth or softness associated with affection."},
        "Infatuation": {"rating": 0, "reasoning": "Neutral gaze does not suggest romantic interest."},
        "Hope/Enthusiasm/Optimism": {"rating": 0, "reasoning": "Face lacks forward-looking energy or specific enthusiasm."},
        "Triumph": {"rating": 0, "reasoning": "No cues of superiority or victory."},
        "Pride": {"rating": 0, "reasoning": "Posture/gaze neutral, not conveying specific pride."},
        "Interest": {"rating": 0, "reasoning": "Gaze is direct but may not indicate active curiosity or fascination beyond acknowledging the camera."},
        "Awe": {"rating": 0, "reasoning": "Neutral expression lacks wonder or amazement cues."},
        "Astonishment/Surprise": {"rating": 0, "reasoning": "Features are calm and neutral, no signs of shock."},
        "Concentration": {"rating": 0, "reasoning": "No signs of intense focus or mental effort."},
        "Contemplation": {"rating": 0, "reasoning": "Expression isn't withdrawn or indicative of deep thought."},
        "Relief": {"rating": 0, "reasoning": "No signs of released tension."},
        "Longing": {"rating": 0, "reasoning": "Gaze is present, not wistful or yearning."},
        "Teasing": {"rating": 0, "reasoning": "Neutral expression lacks playful or provocative cues."},
        "Impatience and Irritability": {"rating": 0, "reasoning": "Face appears relaxed, no tension suggesting irritation."},
        "Sexual Lust": {"rating": 0, "reasoning": "Neutral expression lacks cues for sexual desire."},
        "Doubt": {"rating": 0, "reasoning": "Expression conveys neutrality, not skepticism or uncertainty."},
        "Fear": {"rating": 0, "reasoning": "Face is calm, lacks fear indicators."},
        "Distress": {"rating": 0, "reasoning": "No signs of worry, anxiety, or anguish."},
        "Confusion": {"rating": 0, "reasoning": "Expression is clear and direct, not bewildered."},
        "Embarrassment": {"rating": 0, "reasoning": "Direct gaze and neutral pose lack shyness or awkwardness cues."},
        "Shame": {"rating": 0, "reasoning": "Neutral, direct expression, no signs of guilt or humiliation."},
        "Disappointment": {"rating": 0, "reasoning": "Features are neutral, not downturned."},
        "Sadness": {"rating": 0, "reasoning": "No visible signs of sadness."},
        "Bitterness": {"rating": 0, "reasoning": "Neutral expression lacks resentment cues."},
        "Contempt": {"rating": 0, "reasoning": "Symmetrical features, no sneer or disdain."},
        "Disgust": {"rating": 0, "reasoning": "Neutral expression lacks revulsion cues."},
        "Anger": {"rating": 0, "reasoning": "Face is relaxed, no anger signs."},
        "Malevolence/Malice": {"rating": 0, "reasoning": "Neutral expression lacks spiteful cues."},
        "Sourness": {"rating": 0, "reasoning": "Expression is neutral, not tart or acerbic."},
        "Pain": {"rating": 0, "reasoning": "No grimacing or other pain indicators."},
        "Helplessness": {"rating": 0, "reasoning": "Neutral pose doesn't suggest powerlessness."},
        "Fatigue/Exhaustion": {"rating": 0, "reasoning": "Eyes appear alert (or neutral), no clear signs of weariness."},
        "Intoxication/Altered States of Consciousness": {"rating": 0, "reasoning": "Appearance seems normal and standard."},
        "Jealousy & Envy": {"rating": 0, "reasoning": "Neutral expression lacks comparative or covetous cues."}
        # (INFO: 2 non-zero ratings - constraint met: Contentment, Emotional Numbness)
    },
    "extra_dimensions": {
        "Arousal": {"rating": 2, "reasoning": "Calm, relaxed state. Awake but low energy presentation."},
        "Valence": {"rating": 0, "reasoning": "Expression is neutral, conveying neither positive nor negative affect strongly."},
        "Dominance": {"rating": 0, "reasoning": "Appears balanced, neither particularly assertive/dominant nor submissive."},
        "Emotional Vulnerability": {"rating": 2, "reasoning": "The neutral, possibly slightly guarded expression suggests moderate control and limited emotional openness."}
    }
}
"""

# --- Prompt Definitions ---

# Construct the final prompt string with detailed instructions and examples
NEW_PROMPT_TEMPLATE = f"""
**TASK DEFINITION:**
Analyze the provided image and assess the perceived emotional state of the person depicted. You must evaluate **40 standard emotion dimensions** AND **4 extra dimensions** (Arousal, Valence, Dominance, Emotional Vulnerability). For **ALL 44 dimensions**, provide a numerical **rating** and textual **reasoning** based *only* on visual evidence.

**1. STANDARD EMOTION TAXONOMY (40 Dimensions):**
Rate each dimension on a scale of **0 (Not Present) to 7 (Extremely Strongly Present)**.
{taxonomy_list_string}

**>>> ABSOLUTELY CRITICAL CONSTRAINT <<<**
For the 40 standard emotions listed above: You **MUST** assign a non-zero rating (1-7) to **AT MOST FIVE (5)** dimensions that you perceive most strongly. Choose only the top 1 to 5 emotions that best represent the visual evidence. **ALL OTHER 35+ STANDARD EMOTION DIMENSIONS MUST receive a rating of EXACTLY 0.** There are **NO EXCEPTIONS** to this rule. Failure to adhere means the output is invalid.

**2. EXTRA DIMENSIONS (4 Dimensions):**
Rate these dimensions based on their specific scales provided below. The "Top 5 Non-Zero" constraint **DOES NOT APPLY** to these 4 extra dimensions.
{extra_dims_string}

**REASONING REQUIREMENT:**
For **ALL 44 dimensions (both standard and extra)**, provide concise reasoning (1-3 sentences) for your rating. **Ground your reasoning in specific, observable visual features** in the image (e.g., "Corners of lips turned down," "Eyebrows raised and arched," "Gaze averted," "Jaw clenched," "Shoulders slumped," "Eyes wide," "No visible muscle tension"). Even for ratings of 0, briefly state *why* based on visual evidence (e.g., "Face is neutral, no smile lines visible," "Brow is relaxed, unlike in anger"). Be factual and avoid fluff.

**REQUIRED OUTPUT FORMAT:**
You **MUST** output your analysis as a single, complete Python dictionary object. **NO OTHER TEXT BEFORE OR AFTER.**

*   The dictionary must have exactly two top-level keys: `"emotions"` and `"extra_dimensions"`.
*   The value for `"emotions"` must be a dictionary containing exactly **40** keys (the standard emotion names).
*   The value for `"extra_dimensions"` must be a dictionary containing exactly **4** keys (Arousal, Valence, Dominance, Emotional Vulnerability).
*   The value associated with **each** of the 44 dimension keys (under both "emotions" and "extra_dimensions") **MUST** be *another* dictionary containing exactly two keys:
    1.  `"rating"`: An integer matching the specified scale (0-7 for emotions, **strictly respecting the Top-5 non-zero rule**; specific scales for extra dimensions).
    2.  `"reasoning"`: A string explaining the rating based on visual evidence.

**OUTPUT STRUCTURE EXAMPLE (Illustrative Snippet):**
Your final output **MUST EXACTLY** match this structure.

{{
    "emotions": {{
        "Amusement": {{"rating": 6, "reasoning": "Clear smile, eyes crinkled..."}},
        "Elation": {{"rating": 4, "reasoning": "Broad smile suggests joy..."}},
        # ... other emotions, MOST RATED 0 ...
        "Sadness": {{"rating": 0, "reasoning": "No downturned mouth corners or downcast eyes."}},
        # ... all 40 emotions included, strictly respecting top-5 non-zero rule ...
        "Jealousy & Envy": {{"rating": 0, "reasoning": "Expression is open, lacks envious cues."}}
    }},
    "extra_dimensions": {{
        "Arousal": {{"rating": 5, "reasoning": "High energy suggested by..."}},
        "Valence": {{"rating": 2, "reasoning": "Clearly positive affect shown..."}},
        "Dominance": {{"rating": 1, "reasoning": "Appears confident..."}},
        "Emotional Vulnerability": {{"rating": 3, "reasoning": "Genuine expression suggests openness..."}}
    }}
}}

**PROCESS & FINAL CHECK:**
1.  Carefully analyze the image.
2.  For **each** of the 40 standard emotions: Assign rating 0-7 and provide reasoning. Ensure **ABSOLUTELY NO MORE THAN 5** non-zero ratings are given. Set all others strictly to 0.
3.  For **each** of the 4 extra dimensions: Assign rating based on its scale and provide reasoning.
4.  Construct the final Python dictionary exactly as specified.
5.  **Before generating the final dictionary, double-check that the 'emotions' dictionary adheres strictly to the maximum 5 non-zero rating constraint.**
6.  **Output ONLY the complete Python dictionary.** No introduction, no explanation outside reasoning fields, no markdown code fences.

{MULTI_SHOT_EXAMPLES}

Provide the complete dictionary output now for the given image. **Remember: Exactly "emotions" and "extra_dimensions" keys, with all 44 sub-dimensions having {{"rating": N, "reasoning": "..."}}, and **MOST IMPORTANTLY** respecting the **strict Top-5 non-zero rule** for the 40 standard emotions.**
"""

# "Aha" prompt updated to strongly re-emphasize the Top-5 constraint
ADDITIONAL_PROMPT = """
Wait, wait. Wait. That’s an aha moment. I must flag you here because you are not following my instructions precisely, likely regarding the **CRITICAL CONSTRAINT** about the standard emotion ratings.
Please review the instructions AGAIN, especially the section: **>>> ABSOLUTELY CRITICAL CONSTRAINT <<<**

Your output **MUST** be a single Python dictionary object: `{ "emotions": {...}, "extra_dimensions": {...} }`.
Within the `"emotions"` dictionary (which MUST contain all 40 standard emotion keys):
- **NO MORE THAN FIVE (5) emotions can have a "rating" value greater than 0.**
- **ALL OTHER 35+ standard emotions MUST have "rating": 0.**

This constraint is the most common reason for validation failure. Please meticulously check your proposed ratings for the 40 standard emotions and ensure you have assigned a rating of 0 to all but the top 1-5 most relevant ones based on the image.

Provide the corrected, complete dictionary output now, adhering strictly to ALL constraints, especially the Top-5 non-zero rule for standard emotions. Only the dictionary.
"""

# --- Gemini API Details ---
API_ENDPOINT = 'https://api.hyprlab.io/v1/chat/completions' 
headers = {
    'Authorization': f'Bearer {API_KEY}',
    # 'Content-Type' is omitted to let 'requests' library set it appropriately
}


# --- Thread-Safe State Management ---
processed_count_lock = threading.Lock()
failed_list_lock = threading.Lock()
file_write_locks = {}
file_write_locks_lock = threading.Lock()

# Global counters and lists (protected by locks)
processed_files_count = 0
skipped_files_count = 0
failed_items = []

# --- Helper Functions ---

def get_file_lock(file_path):
    with file_write_locks_lock:
        if file_path not in file_write_locks:
            file_write_locks[file_path] = threading.Lock()
        return file_write_locks[file_path]

def create_prompt(prompt_template, additional_message=""):
    prompt_text = prompt_template
    if additional_message:
        example_marker = "**MULTI-SHOT EXAMPLES"
        marker_pos = prompt_text.find(example_marker)
        if marker_pos != -1:
             prompt_text = prompt_text[:marker_pos] + additional_message + "\n\n" + prompt_text[marker_pos:]
        else:
             prompt_text += "\n\n" + additional_message
    return prompt_text

def validate_annotation(data, image_path_for_log):
    errors = []
    if not isinstance(data, dict):
        errors.append("Top level output is not a dictionary.")
        return errors

    if "emotions" not in data: errors.append("Missing required key: 'emotions'")
    if "extra_dimensions" not in data: errors.append("Missing required key: 'extra_dimensions'")
    expected_top_level_keys = {"emotions", "extra_dimensions"}
    found_top_level_keys = set(data.keys())
    if found_top_level_keys != expected_top_level_keys:
         errors.append(f"Expected keys {expected_top_level_keys}, found {found_top_level_keys}.")


    if "emotions" in data and isinstance(data["emotions"], dict):
        emotions_dict = data["emotions"]
        found_emotion_keys = set(emotions_dict.keys())
        if found_emotion_keys != REQUIRED_EMOTION_KEYS:
            missing = REQUIRED_EMOTION_KEYS - found_emotion_keys
            extra = found_emotion_keys - REQUIRED_EMOTION_KEYS
            if missing: errors.append(f"Missing required emotion keys ({len(missing)}): {sorted(list(missing))[:5]}...")
            if extra: errors.append(f"Found unexpected extra keys under 'emotions' ({len(extra)}): {sorted(list(extra))[:5]}...")

        non_zero_emotion_count = 0
        for dim_name, inner_dict in emotions_dict.items():
            if dim_name not in REQUIRED_EMOTION_KEYS: continue
            if not isinstance(inner_dict, dict): errors.append(f"Value for emotion '{dim_name}' is not a dictionary."); continue
            if "rating" not in inner_dict: errors.append(f"Missing 'rating' key for emotion '{dim_name}'.")
            if "reasoning" not in inner_dict: errors.append(f"Missing 'reasoning' key for emotion '{dim_name}'.")
            if "reasoning" in inner_dict and not isinstance(inner_dict.get("reasoning"), str): errors.append(f"'reasoning' for emotion '{dim_name}' is not a string (found {type(inner_dict.get('reasoning')).__name__}).")

            if "rating" in inner_dict:
                rating_val = inner_dict.get("rating")
                if isinstance(rating_val, float) and rating_val.is_integer():
                    try: rating_val = int(rating_val); inner_dict["rating"] = rating_val
                    except (ValueError, TypeError): errors.append(f"'rating' for emotion '{dim_name}' float conversion failed."); continue
                elif not isinstance(rating_val, int): errors.append(f"'rating' for emotion '{dim_name}' is not an integer (found {type(rating_val).__name__})."); continue

                if isinstance(rating_val, int):
                    if not (0 <= rating_val <= 7): errors.append(f"'rating' for emotion '{dim_name}' ({rating_val}) outside 0-7 range.")
                    if rating_val > 0: non_zero_emotion_count += 1

        if non_zero_emotion_count > 5: errors.append(f"Constraint Violation: Found {non_zero_emotion_count} non-zero ratings in 'emotions', maximum allowed is 5.")
    elif "emotions" in data:
        errors.append("'emotions' value is not a dictionary.")


    if "extra_dimensions" in data and isinstance(data["extra_dimensions"], dict):
        extra_dict = data["extra_dimensions"]
        found_extra_keys = set(extra_dict.keys())
        if found_extra_keys != REQUIRED_EXTRA_DIMENSION_KEYS:
            missing = REQUIRED_EXTRA_DIMENSION_KEYS - found_extra_keys
            extra = found_extra_keys - REQUIRED_EXTRA_DIMENSION_KEYS
            if missing: errors.append(f"Missing required extra_dimension keys ({len(missing)}): {sorted(list(missing))}")
            if extra: errors.append(f"Found unexpected extra keys under 'extra_dimensions' ({len(extra)}): {sorted(list(extra))}")

        for dim_name, inner_dict in extra_dict.items():
             if dim_name not in REQUIRED_EXTRA_DIMENSION_KEYS: continue
             if not isinstance(inner_dict, dict): errors.append(f"Value for extra_dimension '{dim_name}' is not a dictionary."); continue
             if "rating" not in inner_dict: errors.append(f"Missing 'rating' key for extra_dimension '{dim_name}'.")
             if "reasoning" not in inner_dict: errors.append(f"Missing 'reasoning' key for extra_dimension '{dim_name}'.")
             if "reasoning" in inner_dict and not isinstance(inner_dict.get("reasoning"), str): errors.append(f"'reasoning' for extra_dimension '{dim_name}' is not a string (found {type(inner_dict.get('reasoning')).__name__}).")

             if "rating" in inner_dict:
                rating_val = inner_dict.get("rating")
                if isinstance(rating_val, float) and rating_val.is_integer():
                    try: rating_val = int(rating_val); inner_dict["rating"] = rating_val
                    except (ValueError, TypeError): errors.append(f"'rating' for extra_dimension '{dim_name}' float conversion failed."); continue
                elif not isinstance(rating_val, int): errors.append(f"'rating' for extra_dimension '{dim_name}' is not an integer (found {type(rating_val).__name__})."); continue

                if isinstance(rating_val, int):
                    scale_details = EXTRA_DIMENSIONS.get(dim_name)
                    if scale_details and 'scale' in scale_details:
                        try:
                            scale_str = scale_details['scale']
                            if 'to' in scale_str:
                                scale_min, scale_max = map(int, scale_str.split(' to '))
                                if not (scale_min <= rating_val <= scale_max):
                                    errors.append(f"'rating' for {dim_name} ({rating_val}) outside {scale_str} range.")
                            elif '-' in scale_str: # Handling cases like '0-7'
                                scale_min, scale_max = map(int, scale_str.split('-'))
                                if not (scale_min <= rating_val <= scale_max):
                                     errors.append(f"'rating' for {dim_name} ({rating_val}) outside {scale_str} range.")
                            else:
                                errors.append(f"Could not parse scale format for {dim_name}: {scale_str}")

                        except ValueError:
                            errors.append(f"Could not parse scale range integers for {dim_name}: {scale_details['scale']}")
                    elif not scale_details:
                         errors.append(f"Internal error: Scale details not found for validation of '{dim_name}'")

    elif "extra_dimensions" in data:
         errors.append("'extra_dimensions' value is not a dictionary.")

    return errors


def should_process_image(json_path, current_model_name):
    if not json_path.exists():
        return True

    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        if not isinstance(data, list):
            print(f"   [INFO] Corrupted JSON (not a list) found for {json_path.name}. Will overwrite.")
            return True
        for annotation in data:
            if isinstance(annotation, dict) and annotation.get("model") == current_model_name:
                return False
        return True
    except json.JSONDecodeError:
        print(f"   [INFO] Corrupted JSON (decode error) found for {json_path.name}. Will overwrite.")
        return True
    except IOError as e:
        print(f"   [WARN] Error reading JSON file {json_path}: {e}. Assuming needs processing.")
        return True
    except Exception as e:
        print(f"   [WARN] Unexpected error checking JSON {json_path}: {e}. Assuming needs processing.")
        return True


#add by gollam ......................
def extract_content(result):
    """
    Extracts 'content' from API response or returns the whole response if 'content' is missing or improperly structured.
    """
    if isinstance(result, dict):
        # If the result is a dictionary and has a 'content' field, return it. Otherwise, return the whole dictionary.
        content = result.get("content", result)  
        return content
    elif isinstance(result, str):
        try:
            # Try parsing the string as JSON if it's a valid string
            parsed = json.loads(result)
            content = parsed.get("content", parsed)  # Return content if exists, otherwise return the whole parsed object.
            return content
        except json.JSONDecodeError:
            print("   [DEBUG] Could not decode string as JSON.")
            return None
    else:
        print(f"   [DEBUG] extract_content received unsupported type: {type(result)}")
        return None


def extract_content_as_json(response: dict) -> dict:
    """
    Extracts the 'content' JSON from the given chat completion API response.

    Parameters:
        response (dict): The raw API response (already parsed from JSON).

    Returns:
        dict: Parsed JSON object from the 'content' field.
    
    Raises:
        KeyError, IndexError, json.JSONDecodeError: If structure is invalid or content is not valid JSON.
    """
    try:
        # Step 1: Navigate to the 'content' string
        content_str = response['choices'][0]['message']['content']
        
        # Step 2: Parse the content string as JSON
        content_json = json.loads(content_str)
        
        return content_json
    
    except (KeyError, IndexError) as e:
        raise KeyError("Missing expected keys in the response structure.") from e
    except json.JSONDecodeError as je:
        raise ValueError("The 'content' field is not valid JSON.") from je


def generate_json_annotation(img_path, prompt_text=None):
    import mimetypes
    from pathlib import Path
    import requests
    import base64

    if prompt_text is None:
        prompt_text = create_prompt(NEW_PROMPT_TEMPLATE)

    # Open image and encode as base64
    with open(img_path, 'rb') as f:
        encoded_image = base64.b64encode(f.read()).decode('utf-8')

    # Guess the mime type
    mime_type, _ = mimetypes.guess_type(img_path)
    mime_type = mime_type or "image/png"

    filename = Path(img_path).name

    data = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt_text}],
        "files": [{
            "name": filename,
            "data": encoded_image,
            "mime_type": mime_type
        }],
        "max_tokens": 2000
    }

    try:
        # Sending the request to the API
        response = requests.post(API_ENDPOINT, headers=headers, json=data)
        
        # Check if the response is valid (successful HTTP status)
        if not response.ok:
            raise ValueError(f"Request failed with status code {response.status_code}: {response.text}")

        result = response.json()

        # Print result for debugging
        print("............................................")
        #print(result)
        print(".........................................")
        cintent_json = extract_content_as_json(result)

        #print(cintent_json)

        json_output = extract_content(cintent_json)

        if json_output is None:
            print(f"   [DEBUG] No content found in response for {filename}.")
        else:
            print(json_output)

        return json_output

    except requests.exceptions.RequestException as e:
        print(f"[ERROR] Request failed for {filename}: {e}")
    except ValueError as ve:
        print(f"[ERROR] Value error (possibly empty response or invalid output) for {filename}: {ve}")
    except Exception as ex:
        print(f"[ERROR] Unexpected error processing {filename}: {ex}")

    return None


def process_image_worker(image_path_str, json_output_path_str):
    """
    Worker function that processes images and attempts to generate valid annotations.
    Saves a single JSON object to the specified output file with custom format:
    {
        "key": ["source-folder", "model.name", "image.png"],
        "predictions": { ... }
    }
    """
    global processed_files_count, failed_items
    image_path = Path(image_path_str)
    json_path = Path(json_output_path_str)
    filename = image_path.name
    thread_id = threading.get_ident()

    final_valid_annotation = None
    attempt = 0
    max_attempts = 1
    success = False

    # Ensure output directory exists
    json_path.parent.mkdir(parents=True, exist_ok=True)

    if not image_path.exists():
        print(f"   [T-{thread_id}] ERROR: Image {filename} does not exist. Skipping.")
        failed_items.append((str(image_path), "Image Not Found"))
        return False

    if image_path.suffix.lower() not in ['.png', '.jpg', '.jpeg', '.bmp']:
        print(f"   [T-{thread_id}] ERROR: Unsupported image format for {filename}. Skipping.")
        failed_items.append((str(image_path), "Unsupported Image Format"))
        return False

    while attempt < max_attempts:
        current_prompt_text = create_prompt(NEW_PROMPT_TEMPLATE, ADDITIONAL_PROMPT if attempt > 0 else "")
        try:
            raw_response = generate_json_annotation(image_path, current_prompt_text)
            if raw_response is None:
                print(f"   [T-{thread_id}] ERROR: Model returned None for {filename}. Skipping this file.")
                break

            if not isinstance(raw_response, (str, dict)):
                raise TypeError(f"Response is not dict or str: {type(raw_response)}")

            parsed_annotation = extract_content(raw_response)
            if not parsed_annotation:
                print(f"   [T-{thread_id}] DEBUG: extract_content failed. Raw response:\n{raw_response}")
                raise ValueError("No content extracted from model response.")

            if 'emotions' not in parsed_annotation or not isinstance(parsed_annotation['emotions'], dict):
                parsed_annotation['emotions'] = {}

            if 'extra_dimensions' not in parsed_annotation or not isinstance(parsed_annotation['extra_dimensions'], dict):
                parsed_annotation['extra_dimensions'] = {}

            required_keys = ['Arousal', 'Dominance', 'Emotional Vulnerability', 'Valence']
            for key in required_keys:
                if key not in parsed_annotation['extra_dimensions']:
                    parsed_annotation['extra_dimensions'][key] = None

            validation_errors = validate_annotation(parsed_annotation, str(image_path))
            if not validation_errors:
                final_valid_annotation = parsed_annotation
                success = True
                break
            else:
                print(f"\n   [T-{thread_id}] INVALID (Attempt {attempt + 1}) for {filename}:")
                for error in validation_errors:
                    print(f"      - {error}")

        except ValueError as ve:
            print(f"   [T-{thread_id}] ERROR: Validation or model issue for {filename}: {ve}")
        except TypeError as te:
            print(f"   [T-{thread_id}] ERROR: Invalid type in response for {filename}: {te}")
        except Exception as e:
            print(f"   [T-{thread_id}] ERROR: Unexpected error for {filename}: {type(e).__name__} - {e}")

        attempt += 1
        if attempt < max_attempts and not success:
            time.sleep(2 ** attempt)

    if success and final_valid_annotation:
        new_entry = {
            "key": [
                image_path.parent.name,
                MODEL_NAME.replace("-", "."),
                filename
            ],
            "predictions": final_valid_annotation
        }

        file_lock = get_file_lock(str(json_path))
        with file_lock:
            try:
                with open(json_path, 'w', encoding='utf-8') as json_file:
                    json.dump(new_entry, json_file, indent=2, ensure_ascii=False)
                with processed_count_lock:
                    processed_files_count += 1
                return True
            except IOError as io_err:
                print(f"   [T-{thread_id}] ERROR: Could not write JSON file {json_path}: {io_err}")
                with failed_list_lock:
                    failed_items.append((str(image_path), f"Save Error: {io_err}"))
            except Exception as save_err:
                print(f"   [T-{thread_id}] ERROR: Unexpected error saving {json_path}: {save_err}")
                with failed_list_lock:
                    failed_items.append((str(image_path), f"Save Error: {type(save_err).__name__}"))
    else:
        with failed_list_lock:
            failed_items.append((str(image_path), "Validation/API Error/Safety Block"))

    return False


In [70]:

if __name__ == "__main__":
    print(f"Starting recursive image processing from: {INPUT_FOLDER}")
    print(f"Saving outputs to: {OUTPUT_FOLDER}")  # NEW
    print(f"Using model: {MODEL_NAME}")
    print(f"Using {NUM_THREADS} parallel threads.")
    if API_KEY == "YOUR_GEMINI_API_KEY_HERE" or not API_KEY:
        print("\n*** FATAL ERROR: API_KEY is not set. Please update the script! ***\n")
        exit()

    input_path = Path(INPUT_FOLDER)
    output_path = Path(OUTPUT_FOLDER)   # NEW
    input_path.mkdir(parents=True, exist_ok=True)
    output_path.mkdir(parents=True, exist_ok=True)  # NEW

    all_image_paths = []
    print("Discovering image files...")
    for root, _, files in os.walk(input_path):
        root_path = Path(root)
        for file in files:
            if file.lower().endswith(SUPPORTED_EXTENSIONS):
                all_image_paths.append(str(root_path / file))
    print(f"Found {len(all_image_paths)} total image files.")

    images_to_process = []
    print("Checking existing annotations...")
    for img_path_str in all_image_paths:
        img_path = Path(img_path_str)

        # NEW: Create output json path relative to OUTPUT_FOLDER
        relative_img_path = img_path.relative_to(input_path)
        json_out_path = output_path / relative_img_path.with_suffix('.json')
        json_out_path.parent.mkdir(parents=True, exist_ok=True)  # Ensure output subfolder exists

        if should_process_image(json_out_path, MODEL_NAME):  # CHANGED
            images_to_process.append((img_path_str, str(json_out_path)))  # PASS BOTH PATHS
        else:
            skipped_files_count += 1

    total_to_process_count = len(images_to_process)
    print(f"Skipped {skipped_files_count} images (already annotated by {MODEL_NAME}).")
    if total_to_process_count > 0:
        print(f"Submitting {total_to_process_count} images for processing...")
    else:
        print("No images require processing for the specified model.")


    if total_to_process_count > 0:
        futures = []
        start_time = time.time()
        with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
            for img_path_str, json_out_path_str in images_to_process:
                futures.append(executor.submit(process_image_worker, img_path_str, json_out_path_str))  # UPDATED CALL

            completed_count = 0
            for future in as_completed(futures):
                completed_count += 1
                print(f"Progress: {completed_count}/{total_to_process_count} tasks completed.", end='\r', flush=True)
                try:
                    future.result()
                except Exception as exc:
                    print(f'\n   [MAIN] Task generated an uncaught exception: {exc}')

        end_time = time.time()
        print("\nProcessing finished.")


Starting recursive image processing from: /nfs/home/rabbyg/Emotion_detection/stance-detection-part-10
Saving outputs to: /nfs/home/rabbyg/Emotion_detection/JSON
Using model: claude-3-7-sonnet-latest
Using 50 parallel threads.
Discovering image files...
Found 10 total image files.
Checking existing annotations...
Skipped 0 images (already annotated by claude-3-7-sonnet-latest).
Submitting 10 images for processing...
............................................
.........................................
{'emotions': {'Amusement': {'rating': 0, 'reasoning': 'No smile or laughter visible; expression is neutral-to-concerned rather than amused.'}, 'Elation': {'rating': 0, 'reasoning': 'No signs of joy or excitement; expression appears contemplative rather than joyful.'}, 'Pleasure/Ecstasy': {'rating': 0, 'reasoning': 'Expression lacks any indicators of pleasure; appears more serious and thoughtful.'}, 'Contentment': {'rating': 0, 'reasoning': 'Face shows slight tension, not relaxation or sati

In [71]:
import os
import shutil

def sync_folders(folder1, folder2, folder3):
    os.makedirs(folder3, exist_ok=True)

    # Get sets of filenames without extensions
    png_files = {os.path.splitext(f)[0] for f in os.listdir(folder1) if f.endswith('.png')}
    json_files = {os.path.splitext(f)[0] for f in os.listdir(folder2) if f.endswith('.json')}

    # Find mismatched files
    missing_json = png_files - json_files
    missing_png = json_files - png_files

    # Copy missing JSONs
    for name in missing_json:
        src = os.path.join(folder1, name + '.png')
        dst = os.path.join(folder3, name + '.png')
        if os.path.exists(src):
            shutil.copy(src, dst)
            print(f"Copied missing JSON's PNG: {dst}")

    # Copy missing PNGs
    for name in missing_png:
        src = os.path.join(folder2, name + '.json')
        dst = os.path.join(folder3, name + '.json')
        if os.path.exists(src):
            shutil.copy(src, dst)
            print(f"Copied missing PNG's JSON: {dst}")

# Example usage:
# sync_folders("path/to/folder1", "path/to/folder2", "path/to/folder3")
#claude-3-7-sonnet-latest


In [72]:
sync_folders("/nfs/home/rabbyg/Emotion_detection/stance-detection-part-10",
              "/nfs/home/rabbyg/Emotion_detection/JSON",
                "/nfs/home/rabbyg/Emotion_detection/failed_1")

Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_416.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_287.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_462.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_436.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_471.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_261.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_289.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_282.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_499.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image

Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_380.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_381.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_320.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_379.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_332.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_391.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_411.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_401.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image_467.json
Copied missing PNG's JSON: /nfs/home/rabbyg/Emotion_detection/failed_1/flux_prompt_10-image